In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pycaret
from pycaret.classification import *
from pycaret.regression import *
from pycaret.datasets import get_data

In [3]:
data_pm = pd.read_csv('data_predict.csv')
data_pm

,DATETIMEDATA,PM25,O3,WS,TEMP,RH,WD
0,2024-01-01 00:00:00,16.5,16.000000,1,27.0,79.0,115.0
1,2024-01-01 01:00:00,33.6,11.044944,1,27.0,82.0,97.0
2,2024-01-01 02:00:00,34.2,11.044944,1,26.0,84.0,96.0
3,2024-01-01 03:00:00,20.1,11.044944,1,26.0,82.0,108.0
4,2024-01-01 04:00:00,16.5,1.000000,1,27.0,79.0,106.0
...,...,...,...,...,...,...,...
1672,2024-03-10 16:00:00,25.3,30.000000,1,31.0,53.0,194.0
1673,2024-03-10 17:00:00,23.9,28.000000,1,30.0,57.0,214.0
1674,2024-03-10 18:00:00,24.1,27.000000,1,29.0,63.0,202.0
1675,2024-03-10 19:00:00,28.6,4.000000,1,29.0,64.0,138.0


In [4]:
data_pm['DATETIMEDATA'] = pd.to_datetime(data_pm['DATETIMEDATA'])

In [5]:
regression_setup = setup(data_pm, target='PM25', session_id=123, date_features=['DATETIMEDATA'], normalize=True)

,Description,Value
0,Session id,123
1,Target,PM25
2,Target type,Regression
3,Original data shape,"(1677, 7)"
4,Transformed data shape,"(1677, 9)"
5,Transformed train set shape,"(1173, 9)"
6,Transformed test set shape,"(504, 9)"
7,Numeric features,5
8,Date features,1
9,Preprocess,True


In [6]:
best = compare_models()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:36:02
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Linear Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
et,Extra Trees Regressor,3.2966,19.0728,4.3498,0.3220,0.2238,0.1951,0.0460
catboost,CatBoost Regressor,3.3956,19.4903,4.4032,0.3111,0.2289,0.2022,0.3490
rf,Random Forest Regressor,3.3953,19.6079,4.4144,0.3029,0.2291,0.2029,0.0570
gbr,Gradient Boosting Regressor,3.5045,20.4243,4.5073,0.2851,0.2366,0.2117,0.0220
lightgbm,Light Gradient Boosting Machine,3.4638,20.5269,4.5212,0.2718,0.2341,0.2058,0.0870
knn,K Neighbors Regressor,3.6733,23.0584,4.7890,0.1907,0.2506,0.2185,0.0120
ada,AdaBoost Regressor,3.9098,23.9809,4.8891,0.1576,0.2619,0.2468,0.0140
ridge,Ridge Regression,3.9744,25.7546,5.0590,0.1056,0.2693,0.2449,0.0070
lar,Least Angle Regression,3.9743,25.7550,5.0590,0.1056,0.2693,0.2449,0.0070
lr,Linear Regression,3.9743,25.7550,5.0590,0.1056,0.2693,0.2449,0.7930


In [8]:
model = create_model('et')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.1174,17.8252,4.2220,0.0370,0.2358,0.2098
1,3.5706,23.2026,4.8169,0.3609,0.2338,0.1931
2,3.1100,16.3374,4.0420,0.4077,0.2149,0.1906
3,3.5303,25.8403,5.0833,0.0762,0.2644,0.2235
4,3.3799,17.5758,4.1923,0.3790,0.2160,0.1959
5,3.6016,22.2976,4.7220,0.3766,0.2231,0.1901
6,3.0145,15.1802,3.8962,0.5294,0.2009,0.1728
7,3.0035,14.6729,3.8305,0.4761,0.1928,0.1737
8,3.1870,18.2279,4.2694,0.0987,0.2182,0.1857


In [9]:
tuned_model = tune_model(model)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.2504,16.2855,4.0355,0.1202,0.2332,0.2184
1,3.8762,25.6520,5.0648,0.2934,0.2546,0.2203
2,3.5999,19.4904,4.4148,0.2933,0.2417,0.2240
3,3.5410,22.6133,4.7553,0.1916,0.2534,0.2266
4,3.6599,20.4130,4.5181,0.2788,0.2477,0.2257
5,3.9752,25.2957,5.0295,0.2928,0.2475,0.2221
6,3.5928,20.2624,4.5014,0.3718,0.2374,0.2164
7,3.1948,19.0802,4.3681,0.3188,0.2252,0.1943
8,3.3377,19.0787,4.3679,0.0566,0.2300,0.1969


Fitting 10 folds for each of 10 candidates, totalling 100 fits
Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).


In [10]:
model_ensem20 = ensemble_model(tuned_model, n_estimators=20)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.0542,16.0789,4.0099,0.1313,0.2293,0.2059
1,3.5660,22.5795,4.7518,0.3781,0.2312,0.1944
2,3.1929,16.6300,4.0780,0.3971,0.2178,0.1959
3,3.4900,24.8085,4.9808,0.1131,0.2597,0.2225
4,3.2649,16.2580,4.0321,0.4256,0.2121,0.1940
5,3.6212,22.0568,4.6965,0.3834,0.2239,0.1939
6,3.1130,15.8789,3.9848,0.5077,0.2030,0.1773
7,3.0331,15.4947,3.9363,0.4468,0.1988,0.1782
8,3.1475,17.8147,4.2207,0.1191,0.2175,0.1841


In [12]:
last_time = data_pm['DATETIMEDATA'].max()

start_date = last_time + pd.DateOffset(hours=1)
end_date = start_date + pd.DateOffset(days=12, hours=23)  
next_week_hours = pd.date_range(start=start_date, end=end_date, freq='D')

In [13]:
next_data = pd.DataFrame({'DATETIMEDATA': next_week_hours})
next_data['O3'] = 0
next_data['WS'] = 0
next_data['TEMP'] = 0
next_data['RH'] = 0
next_data['WD'] = 0

In [14]:
model_predictions = predict_model(model_ensem20, data=next_data)

In [15]:
mpdf = pd.DataFrame(model_predictions, columns=['DATETIMEDATA', 'prediction_label'])
mpdf = mpdf.rename(columns={'prediction_label': 'PM25'})

print(mpdf)

          DATETIMEDATA       PM25
0  2024-03-10 21:00:00  22.759762
1  2024-03-11 21:00:00  22.821399
2  2024-03-12 21:00:00  22.838767
3  2024-03-13 21:00:00  22.697659
4  2024-03-14 21:00:00  22.495483
5  2024-03-15 21:00:00  22.091482
6  2024-03-16 21:00:00  21.910590
7  2024-03-17 21:00:00  21.791230
8  2024-03-18 21:00:00  21.578651
9  2024-03-19 21:00:00  21.484950
10 2024-03-20 21:00:00  22.185365
11 2024-03-21 21:00:00  22.083616
12 2024-03-22 21:00:00  21.385232


In [16]:
df = pd.DataFrame(mpdf)
df['DATETIMEDATA'] = pd.to_datetime(df['DATETIMEDATA'])
df['DATETIMEDATA'] = df['DATETIMEDATA'].dt.date
df

,DATETIMEDATA,PM25
0,2024-03-10,22.759762
1,2024-03-11,22.821399
2,2024-03-12,22.838767
3,2024-03-13,22.697659
4,2024-03-14,22.495483
5,2024-03-15,22.091482
6,2024-03-16,21.910590
7,2024-03-17,21.791230
8,2024-03-18,21.578651
9,2024-03-19,21.484950


In [17]:
df.to_csv('new_model_predict.csv', index=False)

In [18]:
evaluate_model(model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…